<a href="https://colab.research.google.com/github/Ais2004/Drowsiness/blob/main/Driver_drowsiness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


In [1]:
!rm -rf yolov5
!git clone https://github.com/ultralytics/yolov5


Cloning into 'yolov5'...
remote: Enumerating objects: 17360, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 17360 (delta 36), reused 18 (delta 18), pack-reused 17308 (from 2)
Receiving objects: 100% (17360/17360), 16.25 MiB | 14.38 MiB/s, done.
Resolving deltas: 100% (11901/11901), done.


In [2]:
!pip install -r /content/yolov5/requirements.txt


In [3]:
import cv2
import numpy as np
import torch
import time
from scipy.spatial import distance as dist
from IPython.display import display, clear_output
from matplotlib import pyplot as plt
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-4-5 Python-3.11.11 torch-2.0.1+cu118 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [4]:
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear


In [23]:
# 📤 Upload the photo
from google.colab import files
uploaded = files.upload()


Saving download (4).jpg to download (4).jpg


In [24]:
# 🖼️ Read and display the uploaded image
import cv2
from matplotlib import pyplot as plt

# Check uploaded
print("Uploaded:", uploaded)

# Get the uploaded file name
image_path = list(uploaded.keys())[0]

# Read the image using OpenCV
img = cv2.imread(image_path)
if img is None:
    print("❌ Failed to load image. Check filename or try re-uploading.")
else:
    # Convert BGR to RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Display the image
    plt.figure(figsize=(6,6))
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.title("Uploaded Image")
    plt.show()


Uploaded: {'download (4).jpg': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00\x84\x00\t\x06\x07\x0f\x0f\x0f\x10\r\x0f\x10\x0f\x0f\x0f\x0f\x0f\x0f\r\x0e\x0f\x0f\r\x0f\r\x10\x0f\x10\x15\x11\x16\x17\x15\x15\x16\x16\x18\x1d( \x18\x1d%\x1d\x15\x15!1!%)+...\x17\x1f383,7(-.+\x01\n\n\n\x0e\r\x0e\x17\x10\x10\x17-\x1f\x1e%----+-----------+---------------------+-----------\xff\xc0\x00\x11\x08\x00\xb7\x01\x13\x03\x01\x11\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1c\x00\x00\x02\x03\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x05\x06\x04\x03\x07\x08\xff\xc4\x00>\x10\x00\x01\x04\x00\x04\x03\x06\x02\x07\x06\x05\x05\x00\x00\x00\x00\x01\x00\x02\x03\x11\x04\x12!1\x05AQ\x06\x13"aq\x812\x91\x07\x14B\xa1\xb1\xc1\xd1#Rbr\xe1\xf03Cs\x82\xa2\x15\x16\x17\x92\xb2\xff\xc4\x00\x1b\x01\x01\x01\x00\x03\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\xff\xc4\x006\x11\x01\x00\x02\x02\x00\x04\x03\x04\n\x01\x04\x03\x01\x00\x00\x00\x00

In [25]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-4-5 Python-3.11.11 torch-2.0.1+cu118 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [26]:
# 🧠 Run YOLOv5 model on the image
results = model(img)

# Print detected objects
results.print()

# Draw bounding boxes on image
results.render()

# Display result image with bounding boxes
plt.imshow(results.ims[0])
plt.axis('off')
plt.title("YOLOv5 Detections")
plt.show()


image 1/1: 183x275 1 person
Speed: 29.9ms pre-process, 361.6ms inference, 3.4ms NMS per image at shape (1, 3, 448, 640)


In [11]:
!pip install mediapipe


In [27]:
import mediapipe as mp
import numpy as np

# Eye landmarks index (Mediapipe FaceMesh)
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

# Calculate EAR
def calculate_ear(eye_landmarks):
    # Vertical distances
    A = np.linalg.norm(eye_landmarks[1] - eye_landmarks[5])
    B = np.linalg.norm(eye_landmarks[2] - eye_landmarks[4])
    # Horizontal distance
    C = np.linalg.norm(eye_landmarks[0] - eye_landmarks[3])
    ear = (A + B) / (2.0 * C)
    return ear


In [28]:
mp_face_mesh = mp.solutions.face_mesh

# Load and process image
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Detect facial landmarks
with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1) as face_mesh:
    results = face_mesh.process(image_rgb)

    if results.multi_face_landmarks:
        face_landmarks = results.multi_face_landmarks[0]
        h, w, _ = image.shape

        left_eye = []
        right_eye = []

        for idx in LEFT_EYE:
            x = int(face_landmarks.landmark[idx].x * w)
            y = int(face_landmarks.landmark[idx].y * h)
            left_eye.append(np.array([x, y]))

        for idx in RIGHT_EYE:
            x = int(face_landmarks.landmark[idx].x * w)
            y = int(face_landmarks.landmark[idx].y * h)
            right_eye.append(np.array([x, y]))

        left_ear = calculate_ear(np.array(left_eye))
        right_ear = calculate_ear(np.array(right_eye))
        avg_ear = (left_ear + right_ear) / 2.0

        # Threshold for drowsiness
        EAR_THRESHOLD = 0.25

        if avg_ear < EAR_THRESHOLD:
            print("😴 Drowsy: Eyes are closed")
        else:
            print("🙂 Alert: Eyes are open")
    else:
        print("No face detected.")


🙂 Alert: Eyes are open


In [29]:
# Inner lip landmarks from Mediapipe FaceMesh
MOUTH_LANDMARKS = [13, 14, 78, 308, 82, 312]  # [top, bottom, left, right, upper-left, upper-right]


In [30]:
def calculate_mar(landmarks):
    # Vertical: top (13) to bottom (14)
    A = np.linalg.norm(landmarks[0] - landmarks[1])
    # Horizontal: left (78) to right (308)
    B = np.linalg.norm(landmarks[2] - landmarks[3])
    mar = A / B
    return mar


In [31]:
        # Get mouth landmarks
        mouth = []
        for idx in MOUTH_LANDMARKS:
            x = int(face_landmarks.landmark[idx].x * w)
            y = int(face_landmarks.landmark[idx].y * h)
            mouth.append(np.array([x, y]))

        mar = calculate_mar(mouth)

        # Threshold for yawn
        MAR_THRESHOLD = 0.6

        # Final analysis
        if avg_ear < EAR_THRESHOLD and mar > MAR_THRESHOLD:
            print("😴 Drowsy: Eyes closed + Yawning detected")
        elif avg_ear < EAR_THRESHOLD:
            print("😐 Warning: Eyes closed")
        elif mar > MAR_THRESHOLD:
            print("😮 Yawning detected")
        else:
            print("🙂 Alert: Eyes open and no yawning")


🙂 Alert: Eyes open and no yawning


In [33]:
# Define 3D model points of face landmarks (nose, eyes, chin)
model_points = np.array([
    (0.0, 0.0, 0.0),             # Nose tip
    (0.0, -330.0, -65.0),        # Chin
    (-225.0, 170.0, -135.0),     # Left eye left corner
    (225.0, 170.0, -135.0),      # Right eye right corner
    (-150.0, -150.0, -125.0),    # Left mouth corner
    (150.0, -150.0, -125.0)      # Right mouth corner
], dtype=np.float64)

# Get 2D points from face landmarks
image_points = np.array([
    (face_landmarks.landmark[1].x * w, face_landmarks.landmark[1].y * h),   # Nose tip
    (face_landmarks.landmark[152].x * w, face_landmarks.landmark[152].y * h), # Chin
    (face_landmarks.landmark[33].x * w, face_landmarks.landmark[33].y * h),   # Left eye
    (face_landmarks.landmark[263].x * w, face_landmarks.landmark[263].y * h), # Right eye
    (face_landmarks.landmark[61].x * w, face_landmarks.landmark[61].y * h),   # Left mouth
    (face_landmarks.landmark[291].x * w, face_landmarks.landmark[291].y * h), # Right mouth
], dtype=np.float64)

# Camera matrix and distortion
focal_length = w
center = (w / 2, h / 2)
camera_matrix = np.array([
    [focal_length, 0, center[0]],
    [0, focal_length, center[1]],
    [0, 0, 1]
], dtype="double")

dist_coeffs = np.zeros((4, 1))  # Assume no lens distortion

# Solve for head pose
_, rotation_vector, translation_vector = cv2.solvePnP(
    model_points, image_points, camera_matrix, dist_coeffs)

# Get angles
rmat, _ = cv2.Rodrigues(rotation_vector)
angles, _, _, _, _, _ = cv2.RQDecomp3x3(rmat)

pitch, yaw, roll = angles

if abs(pitch) > 15:
    print("⚠️ Head tilted down - possible drowsiness")
elif abs(yaw) > 20:
    print("👀 Looking away")


⚠️ Head tilted down - possible drowsiness
